In [1]:
import json
import time
import re
import itertools
import numpy as np
import pandas as pd
import datetime as DT
from tqdm import tqdm
from datetime import datetime
from scipy.optimize import curve_fit
from transformers import AutoModelForCausalLM, AutoTokenizer
from awq import AutoAWQForCausalLM
#from sentence_transformers import SentenceTransformer

/data/helulu/miniconda3/envs/jiuan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 设置环境变量
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'

# 模型名称
model_name = "/data/liangyunfei/Qwen2.5-72B-Instruct-AWQ"

# 自动分配设备
model_72Bn = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

# 加载分词器
tokenizer_72Bn = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards: 100%|██████████| 11/11 [00:06<00:00,  1.76it/s]


In [3]:
def chat(prompt):
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer_72Bn.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return model_72Bn.chat(text)


In [4]:
from transformers import AutoTokenizer, AutoModel
def get_embeddings(sentences, model_path='/home/helulu/jiuan_health/bge-large-zh-v1___5'):
    # 加载模型和分词器
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    model.eval()
    
    # 对输入文本进行编码
    encoded_input = tokenizer(
        sentences, 
        padding=True, 
        truncation=True,
        max_length = 512, 
        return_tensors='pt'
    )
    
    # 计算嵌入向量
    with torch.no_grad():
        model_output = model(**encoded_input)
        # 使用 CLS token 的输出作为句子嵌入
        sentence_embeddings = model_output[0][:, 0]
    
    # 归一化嵌入向量
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    
    return sentence_embeddings


In [5]:
from transformers import AutoTokenizer, AutoModel
#创建向量知识库
class KB:
    def __init__(self, file_path):
        self.file_path = file_path
        with open(file_path, 'r', encoding='utf-8') as f:
            self.data = f.read()
        self.docs = self.split_text(self.data)
        self.embeddings = self.encode(self.docs)

    #分词
    def split_text(self, text):
        chunks = [chunk.strip() for chunk in text.split('\n') if chunk.strip()]
        return chunks
    
    #创建向量
    def encode(self,texts):
        embeds = []
        for text in texts:
            response = get_embeddings(text)
            embeds.append(response)
        return np.array(embeds)


kb = KB("/home/helulu/jiuan_health/faiss/cleaned_result.txt")





In [14]:
# 相似度匹配 - 返回前k个最相似的结果
def search(query, kb, top_k=5):
    # 存储所有文档的分数
    scores = []
    e = get_embeddings(query)
    
    # 计算所有文档的相似度分数
    for i, kb_embed in enumerate(kb.embeddings):
        score = float(np.dot(e, kb_embed.T))
        scores.append((score, i))
    
    # 按分数降序排序并获取前top_k个结果
    scores.sort(reverse=True)
    top_results = scores[:top_k]
    
    # 返回结果列表
    results = []
    for score, idx in top_results:
        results.append({
            'score': score,
            'text': kb.docs[idx]
        })
    
    return results

# 使用示例
query = "吃手抓饼可以吗？"
results = search(query, kb)

# 打印结果
print(f"\n查询: {query}")
print("\n相似度最高的5个结果：")
for i, result in enumerate(results, 1):
    print(f"\n{i}. 相似度分数: {result['score']:.4f}")
    print(f"文本: {result['text']}")


查询: 吃手抓饼可以吗？

相似度最高的5个结果：

1. 相似度分数: 0.6819
文本: 手抓饼等高油脂食物，建议尽量少吃，因为高油脂食物会增加胰岛素抵抗，影响血糖控制。

2. 相似度分数: 0.6799
文本: 油脂较大的食物（如手抓饼）不仅热量高，还可能影响血糖控制，建议尽量少吃。

3. 相似度分数: 0.6637
文本: 手抓饼等加工食品通常含有较多的油脂和糖分，容易导致血糖波动。建议尽量少吃这类食物，如果需要加餐，可以选择一些低GI的食物，如全麦面包、燕麦片等。

4. 相似度分数: 0.6434
文本: 手抓饼油脂较大，建议尽量少吃，芋头和手抓饼都是主食，加起来吃拳头大小为宜，午餐需增加蔬菜，蔬菜富含丰富的膳食纤维，可以帮助延缓主食消化吸收，稳定餐后血糖，每顿最好能吃到半斤。

5. 相似度分数: 0.6406
文本: 手抓饼油脂较大，建议尽量少吃，芋头和手抓饼都是主食，加起来吃拳头大小为宜，午餐缺一份蔬菜，注意增加蔬菜，蔬菜富含丰富的膳食纤维，可以帮助延缓主食消化吸收，稳定餐后血糖，每顿最好能吃到半斤。


/tmp/ipykernel_245832/1182251941.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score = float(np.dot(e, kb_embed.T))


In [11]:
#相似度匹配
def search(query,kb):
    max_score = 0
    max_index = 0
    e = get_embeddings(query)
    for i,kb_embed in enumerate(kb.embeddings):
        score = np.dot(e,kb_embed.T)
        if score > max_score:
            max_score = score
            max_index = i
    return kb.docs[max_index]

search("我中午吃的米饭和土豆？",kb)    

'土豆也是主食，如果吃了土豆，米饭就要减量了'

In [16]:

query = "土豆和大米可以一起吃吗"
context = search(query,kb)
prompt_template = """
        基于{context}，回答{query}：不要分点回答，要生成一段文字
        """
prompt = prompt_template.format(context=context,query=query)
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": query}
]
text = tokenizer_72Bn.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer_72Bn([text], return_tensors="pt").to(model_72Bn.device)
generated_ids = model_72Bn  .generate(
    **model_inputs,
    max_new_tokens=256
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer_72Bn.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

/tmp/ipykernel_245832/1182251941.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  score = float(np.dot(e, kb_embed.T))


土豆和大米可以一起吃，但需要注意控制总量，以避免碳水化合物摄入过多。由于土豆属于主食类食物，其淀粉含量较高，与米饭一起食用时，容易导致餐后血糖升高。因此，建议在食用土豆的同时适当减少米饭的摄入量，每餐主食总量最好不超过2两（100克），或者控制在自己拳头大小的范围内。这样既能保证营养均衡，又能避免碳水化合物摄入过量带来的健康问题。
